<a href="https://colab.research.google.com/github/InDl2k/collab/blob/main/%D0%A6%D0%9A_(1_%D0%B7%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D0%B5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

file = pd.read_csv('/content/train.csv')
married = pd.get_dummies(file['ever_married'], drop_first=True)
gender = pd.get_dummies(file['gender'], drop_first=True)
work = pd.get_dummies(file['work_type'])
resident = pd.get_dummies(file['Residence_type'], drop_first=True)
smoke = pd.get_dummies(file['smoking_status'])
file['bmi'].fillna(file['bmi'].mean(), inplace=True)
file.drop(['id', 'ever_married', 'gender', 'work_type', 'Residence_type', 'smoking_status'], axis = 1, inplace = True)
file = pd.concat([file, married, gender, work, resident, smoke], axis=1)

scaler = StandardScaler()
cols_to_scale = ['age', 'avg_glucose_level', 'bmi']
scaler.fit(file[cols_to_scale])
file[cols_to_scale] = scaler.transform(file[cols_to_scale])
file.columns = file.columns.map(str)

X_train = file.drop('stroke', axis=1)
Y_train = file['stroke']

model = LogisticRegression()
model.fit(X_train, Y_train)
LogisticRegression()
y_pred_train = model.predict(X_train)


conf_matrix = confusion_matrix(Y_train, y_pred_train)
conf_matrix_df = pd.DataFrame(conf_matrix)
conf_matrix_labels = pd.DataFrame(conf_matrix, columns =['Прогноз нету инсульта', 'Прогноз есть инсульт'], index = ['Факт нету инсульта', 'Факт есть инсульт'])
conf_matrix_labels

model_accuracy = accuracy_score(Y_train, y_pred_train)
#print(round(model_accuracy, 3))

X_test = pd.read_csv('/content/test.csv')
married = pd.get_dummies(X_test['ever_married'], drop_first=True)
gender = pd.get_dummies(X_test['gender'], drop_first=True)
work = pd.get_dummies(X_test['work_type'])
resident = pd.get_dummies(X_test['Residence_type'], drop_first=True)
smoke = pd.get_dummies(X_test['smoking_status'])
X_test['bmi'].fillna(X_test['bmi'].mean(), inplace=True)
X_test.drop(['id', 'ever_married', 'gender', 'work_type', 'Residence_type', 'smoking_status'], axis = 1, inplace = True)
X_test = pd.concat([X_test, married, gender, work, resident, smoke], axis=1)
X_test.drop(['Other'], axis = 1, inplace = True)


X_test[cols_to_scale] = scaler.transform(X_test[cols_to_scale])
X_test.columns = X_test.columns.map(str)
X_test = X_test.drop('stroke', axis=1)


y_pred_test = model.predict(X_test)


test = pd.read_csv('/content/test.csv')
ids = test['id']
result = pd.DataFrame({'id': ids, 'stroke': y_pred_test})
result['stroke'] = result['stroke'].mask(result['stroke'].eq(0), '0');
result.to_csv("result.csv", sep=',', encoding='utf-8', index=False)